# Geospatial Intelligence: Revisiting Bow at Banff

In the previous homework, we analyzed the output from SUMMA from four different spatial configurations (HRUs+GRUs). In this homework, we are going to dive deeper into how spatial configurations for SUMMA can actually be assembled from environmental data. Our focus in this exercise will be to use elevation data to derive sub-basins (GRUs) in the watershed and then to discretize the height above nearest drainage area within these sub-basins (HRUs). Throughout this exercise you will learn how to:

1. Compute drainage area from elevation data
2. Delineate river networks from the computed drainage area
3. Delineate sub-basins (GRUs) from river networks
4. Compute height above nearest drainage area (HAND)
5. Discretize the height above nearest drainage area (HAND) maps into HAND bands (HRUs)
6. Bring all the previous steps together to code an algorithm to compute HRUs and GRUs for SUMMA
7. Use the constructed algorithm to assemble spatial configurations for SUMMA and then run the model

All problems that have **(Optional)** at the beginning are not required. They are meant primarily for those who want to dig deeper into the Geospatial Intelligence section of the course. Bonus points will be awarded for their completion.

This assignment was designed to work "as is" on HydroShare. If you are running this outside that platform (i.e., a personal laptop) you will need to install a local copy of geospatialtools (among other libraries). To do that you will need to run the following command at the terminal `pip install "git+https://github.com/chaneyn/geospatialtools.git@dev_nate”`. Remember that this is not necessary if you are doing this homework via HydroShare's JupyterHub server.

Let's start by importing some libraries that you will need throughout the assignment

In [ ]:
import geospatialtools.terrain_tools as terrain_tools
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import cartopy
import copy
import xarray as xr
import pysumma as ps
import summa_preprocessing
import subprocess
import glob
import time
import datetime
from hs_restclient import HydroShare 
import zipfile 
import os
%matplotlib inline

Now we'll grab the forcing data from Hydroshare and then we're good to go.

In [ ]:
# Make a hydroshare object - note: needs authentication
hs = HydroShare()

In [ ]:
# Specify the resource ID and download the resource into the main exercise directory
resource_id = 'ed6e22a79b254a33b57b87c39f51136e'
save_here = './'
hs.getResource(resource_id, destination=save_here)

In [ ]:
with zipfile.ZipFile(resource_id+'.zip', 'r') as zip_ref:
    
    # find all files in the zip
    zip_files = zip_ref.namelist()

    # loop over the files and extract only the .nc
    for zip_file in zip_files:
        if zip_file.endswith('.nc'):
            
            # print
            print('Now extracting ' + zip_file)
            
            # extract the file
            zip_ref.extract(zip_file,'tmp_zip')
            
            # find where the file is inside tmp_zip
            for root, dirs, files in os.walk('./tmp_zip'):
                for file in files:
                    file_name = file
                    old_loc = os.path.join(root,file_name)
                    old_folder = root
            
            # Take the file out of the temporary folder and put it where it shoule go
            os.rename(old_loc,'./data/'+file_name)

In [ ]:
# remove the file and the tmp_zip dir
!rm -r 'ed6e22a79b254a33b57b87c39f51136e.zip'
!rm -r './tmp_zip/'

# 0. Initial exploration (0 pts)

The terrain analysis module that you will be using in the homework is called `geospatialtools.terrain_tools`. More specifically, we will be using an object called `terrain_analysis` that will give us all the functions that we need for the majority of this assignment. An instance of the object is created below using the 3 arcsec DEM file from the Bow at Banff. We also create a simple plot of the data.

In [ ]:
#Let's define the network database
file = 'data/dem.tif'
bow = terrain_tools.terrain_analysis(file)

In [ ]:
#Let's see what the instance contains
dir(bow)

In [ ]:
#Let's plot the dem
def plot_dem(data,title):
    fig = plt.figure(figsize=(20,20))
    #proj = projection=cartopy.crs.AlbersEqualArea(standard_parallels=(29.5,45.5),central_longitude=260,central_latitude=45)
    proj = cartopy.crs.PlateCarree()
    ax = plt.subplot(projection=proj)
    tmp = np.ma.masked_array(data.dem,data.dem==-9999)
    bounds = data.bounds
    img_extent = (bounds.left,bounds.right,bounds.bottom,bounds.top)
    cs = ax.imshow(tmp,transform=proj,cmap='terrain',extent=(img_extent),interpolation='nearest')
    cb = plt.colorbar(cs,orientation='horizontal',pad=0.01,shrink=0.9)
    cb.ax.tick_params(labelsize=25)
    plt.title(title,fontsize=40)
    plt.show()
    return
plot_dem(bow,'Elevation (meters)')

# 1. Drainage Area (10 pts)

Drainage area is one of the most used concepts from terrain analysis in hydrology. In essence, it allows us to determine the total accumulated upslope area that drains to a given point. There are multiple ways to do this, with the most common approach being the 8-direction approach. Below we provide an example of how to calculate drainage area using the terrain analysis package.

In [ ]:
#Use the function "calculate drainage area" in terrain_tools
bow.calculate_drainage_area()

**1.1 Create a plot of the accumulation area in log10 space. Note that the drainage area is now within the object `bow` as the variable `acc` (bow.acc). Describe what you observe.**

**1.2 Plot a histogram (plt.hist) of the log10 drainage area values and interpret the results. For consistency, use 25 bins in the histogram.**

In [ ]:
#Here we process the data for direct input into the histogram
tmp = bow.acc.flatten() #
tmp = tmp[tmp != -9999] #

*(Optional)* **1.3 Although there are many software tools to compute drainage area, making your own function that is able to delineate drainage area from a sink-filled DEM is a valuable exercise as it helps provide intuition into terrain analysis algorithms. Using the cell block below, create a function that is able to reproduce the computed drainage area map at Bow at Banff. We strongly recommend using the `numba` package to accelerate your code**.

# 2. River network delineation (15 pts)

The most common approach to delineate river networks from DEMs leverages the precomputed drainage area map. In this exercise, grid cells are classified as channels when their drainage area is above a user-defined threshold, $\tau_{da}$. Below we provide an example of how to delineate the river network for $\tau_{da} = 10^8$ and make a simple plot. 

In [ ]:
def plot_network(data,title):
    fig = plt.figure(figsize=(20,20))
    proj = cartopy.crs.PlateCarree()
    ax = plt.subplot(projection=proj)
    tmp = data.dem
    tmp = np.ma.masked_array(tmp,tmp==-9999)
    bounds = data.bounds
    img_extent = (bounds.left,bounds.right,bounds.bottom,bounds.top)
    cs = ax.imshow(tmp,transform=proj,cmap='terrain',extent=(img_extent),alpha=0.6)
    #Add the river network
    for geom in data.channels_vector:   
     ax.add_geometries([geom,],crs=proj,edgecolor='k',facecolor='none',lw=4,alpha=1.0)
    
    plt.title(title,fontsize=40)
    plt.show()
    return

#Let's set the threshold and delineate the river network
bow.channel_threshold = 10**8 #m2
bow.delineate_river_network()
#Let's plot the corresponding vector river network on top of the DEM
plot_network(bow,r'River network ($\tau_{da} = 10^8\;m^2$)')

**2.1 Construct and plot 4 different river networks by changing $\tau_{da}$ to $2\cdot10^7$, $5\cdot10^7$, $10^8$, and $10^9$.**

**Caution**: Make sure to perform a copy of the instance `bow` using copy.deepcopy. An example of how copy the instance and then change the channel initiation threshold is shown below.

In [ ]:
#Example of using deepcopy
bow_ta8 = copy.deepcopy(bow)
bow_ta8.channel_threshold = 10**8 #m^2

**2.2 A common measure to estimate how well a given basin is drained by stream channels is drainage density. The drainage density is defined as the total length $L$ of streams and rivers in a basin divided by the total area $A$ of the basin ($D_d = \frac{L}{A}$). Compute and compare the drainage density ($km/km^2$) of the the four previously computed river networks. What do we learn about the sensitivity of the river network to $\tau_{da}$? When we compare these values to what is commonly observed values of drainage density, what do we learn?**

An example is provided below for how to extract both $L$ and $A$ from the delineated channels.

In [ ]:
#Here is an example on how to extract the stream length (m) and basin area (m^2) from the object.
area = bow.basin_area #m^2
length = np.sum(bow.reach_length) #m

# 3. Basin delineation (15 pts)

Once we have the river network delineated, we can use the flow direction map to determine the reach that water leaving a given pixel will eventually flow into. By assigning each pixel to its corresponding reach, we are effectively delineating the sub-basins. Below we provide an example of how to do this over Bow at Banff. 

In [ ]:
def plot_basins(data):
    fig = plt.figure(figsize=(20,20))
    proj = projection=cartopy.crs.PlateCarree()
    ax = plt.subplot(projection=proj)
    tmp = data.basins
    tmp = np.ma.masked_array(tmp,tmp<=0)
    bounds = data.bounds
    img_extent = (bounds.left,bounds.right,bounds.bottom,bounds.top)
    cs = ax.imshow(tmp,transform=proj,cmap='prism',extent=(img_extent),alpha=0.6,interpolation='nearest')
    #Add the river network
    for geom in data.channels_vector:   
     ax.add_geometries([geom,],crs=proj,edgecolor='k',facecolor='none',lw=4,alpha=1.0)
    plt.show()
    return
bow.delineate_basins()
plot_basins(bow)

**3.1 Assemble and plot the sub-basin maps that correspond to the four derived river networks from the previous exercise.**

**3.2 Plot the histogram of sub-basins areas for each of the four derived river networks and interpret the results.**

Note that basin area is already computed for each scenario and can be accessed four our example via `bow.basin_area`.

# 4. Height above nearest drainage (15 pts)

With the computed sub-basins, we can now determine the relative elevation difference between a given pixel and the channel into which it flows. This concept is formally known as height above nearest drainage (HAND) and it is increasingly being used to represent hillslopes in semi-distributed models. Below is an example of how this can be done with our baseline instance `bow`.

In [ ]:
def plot_hand(data):
    fig = plt.figure(figsize=(20,20))
    proj = projection=cartopy.crs.PlateCarree()
    ax = plt.subplot(projection=proj)
    tmp = data.hand
    tmp = np.ma.masked_array(tmp,tmp<=0)
    img_extent = (data.bounds.left,data.bounds.right,data.bounds.bottom,data.bounds.top)
    cs = ax.imshow(tmp,transform=proj,cmap='terrain',extent=(img_extent),alpha=0.6,interpolation='nearest')
    #Add the river network
    for geom in data.channels_vector:   
     ax.add_geometries([geom,],crs=proj,edgecolor='k',facecolor='none',lw=4,alpha=1.0)
    plt.show()
    return
bow.calculate_height_above_nearest_drainage()
plot_hand(bow)

**4.1 Assemble and plot the HAND maps that correspond to the derived river networks from the previous exercise. Describe the differences (and similarities) between them.**

**4.2 Plot the histogram of each hand map and comments on the results. If we were to include these HAND maps as a way to represent hillslopes in a hydrologic model, how would the response of the model differ between HAND map?. Below we provide an example of how to remove the undefined pixels (-9999) from the HAND map.**

In [ ]:
tmp = bow.hand[bow.hand!=-9999]

# 5. Discretizing the HAND data (15 pts)

In semi-distributed models, our goal is to use as much information as possible  while minimizing the additional computational burden from a fully distributed model. To this end, we want to simplify our data that we feed into the model to maximize model realism while minimizing computational expense. Here we will do that by discretizing (i.e., binning) the HAND data per sub-basin. We will call these binned HAND values `HAND bands`. In this case, we will be discreziting the HAND data per sub-basin by defining a parameter `dh` which is the difference in average HAND between two adjacent HAND bands. Below is an example of how this can be done with our `bow` instance.

In [ ]:
def plot_height_bands(data):
    fig = plt.figure(figsize=(20,20))
    proj = projection=cartopy.crs.AlbersEqualArea(standard_parallels=(29.5,45.5),central_longitude=260,central_latitude=45)
    ax = plt.subplot(projection=proj)
    tmp = data.hbands
    tmp = np.ma.masked_array(tmp,tmp<=0)
    img_extent = (data.bounds.left,data.bounds.right,data.bounds.bottom,data.bounds.top)
    cs = ax.imshow(tmp,transform=proj,cmap='prism',extent=(img_extent),alpha=0.6,interpolation='nearest')
    #Add the river network
    for geom in data.channels_vector:   
     ax.add_geometries([geom,],crs=proj,edgecolor='k',facecolor='none',lw=4,alpha=1.0)
    plt.show()
    return
            
#Define the desired vertical difference between adjacent height bands
bow.dh = 250 #meters
bow.discretize_hand()
plot_height_bands(bow)

**5.1 Explore how the discretized HAND maps vary when `dh` is set to 5, 20, and 100 m. Interpret the results based on how it will represent the basins/hillslopes in a hydrologic model.**

**5.2 There are multiple ways one can think of discretizing the HAND data. Describe plausible alternatives assuming that we aim to ensure higher spatial detail close to the riparian areas when compared to further upslope. There is not a specific answer that we are looking for here; we want you to use your creativity and describe in words an approach that you think would improve the discretization.**

*(Optional)* **5.3 Code up and try one your proposed approaches to calculate the height bands from the HAND data. Plot the resulting map.**

# 6. Putting it all together (10 pts)

Now that we have the sub-basins (GRUs) and HAND bands (HRUs), we have one approach to derive the spatial setup for SUMMA. Let's now put it all together.

**6.1 Create a function called `assemble_height_bands` that encapsulates in a single function all the different steps that have been identified in this exercise to create the height band maps. Plot the final map of height bands for the case where $\tau_{da} = 10^8$ and $dh = 250$. Note that $\tau_{da}$ and $dh$ should be passed as arguments into the function.**

# 7. Back to Bow: Implementing the derived HRUs and GRUs into SUMMA (30 pts)

Having now gone through the primary steps that lead to defining the spatial configuration of SUMMA, we are going to return to SUMMA to test our developed configurations. More specifically, we will create different spatial configurations based on the pameters $\tau_{da}$ and $dh$ and then use those configurations to run SUMMA between October 1st, 2000 and September 30th, 2001. 

For this exercise, we have preassembled a python library called `summa_preprocessing` that will take your inputs of $\tau_{da}$ and $dh$ and prepare the associated HRUs and GRUs and all the files necessary to run SUMMA and mizuRoute. 

Below we provide an example of how this is done for the case of $\tau_{da} = 10^8$ and $dh = 1000$. We also provide a figure that shows the simulated SWE over the year and the simulated streamflow between 05/01/2001 and 09/23/2001. Note that the simulation will take roughly a minute. 

In [ ]:
tda = 10**8 #tda
dh =  1000 #m
bow = summa_preprocessing.assemble_summa_configuration(tda,dh)
print('ngru',bow.ngru)
print('nhru',bow.nhru)
#Run SUMMA
executable = '/usr/local/bin/summa.exe'
sdir = bow.sdir
print(sdir)
file_manager = '%s/fileManager.txt' % sdir
sim = ps.Simulation(executable, file_manager)
time_start = time.time()
sim.run('local', run_suffix='default')
time_end = time.time()
print(sim.status)
print('SUMMA run took ' + str(round(time_end - time_start)) + ' sec.')
#Run and postprocess mizuRoute
# paths to routing .exe and settings
mizuExe = 'route_runoff.exe'
mizuArg = '%s/mizuroute.control' % sdir
# run mizuRoute through subprocess module
time_start = time.time()
log = subprocess.run([mizuExe, mizuArg], capture_output=True)
time_end = time.time()
print('mizuRoute run took ' + str(round(time_end - time_start)) + ' sec.')
# routing post-processing: find the individual annual mizuRoute output files
files = glob.glob('%s/*flow_routed*' % sdir)
# make sure they are in the right order
files.sort() 
# routing post-processing: merge the annual files into a single file
save_file = '%s/flow_routed.nc' % sdir
ds = xr.merge([xr.open_dataset(file) for file in files]) # merge individual files
ds.to_netcdf(save_file) # save to file

In [ ]:
plt.figure(figsize=(15,7));
sdir = 'summa_sim/tda%.2f_dh%.2f' % (np.log10(tda),dh)
attributes_file = '%s/attributes.nc' % sdir
att_xr = xr.open_dataset(attributes_file)
hru_area = att_xr['HRUarea'][:]
fct = np.array(hru_area/np.sum(hru_area))
output_file = '%s/output_default_timestep.nc' % sdir
sim_xr = xr.open_dataset(output_file)
label = '%s' % sdir.split('/')[-1]
scalarSWE = np.sum(fct*sim_xr.scalarSWE,axis=1)
plt.plot(scalarSWE,label=label,linewidth=7)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.ylabel('simulated SWE [kg m-2]',fontsize=25);
plt.xlabel('');
sim_xr.close()
plt.show()

In [ ]:
plt.figure(figsize=(15,7));
sdir = 'summa_sim/tda%.2f_dh%.2f' % (np.log10(tda),dh)
mroute_file = '%s/flow_routed.nc' % sdir
sim_xr = xr.open_dataset(mroute_file)
label = '%s' % sdir.split('/')[-1]
sim_xr['IRFroutedRunoff'][:,0].plot(label=label,linewidth=5);
plt.legend(fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.ylabel('Discharge at outlet [m3 s-1]',fontsize=25);
plt.xlabel('');
sim_xr.close()
plt.xlim([datetime.datetime(2001,5,1),datetime.datetime(2001,7,15,23)])
plt.show()

**7.1 For the four cases below, run SUMMA and mizuRoute. Document the runtime of SUMMA for each case.**
* $\tau_{da} = 10^9$, $dh = 2000$
* $\tau_{da} = 10^8$, $dh = 1000$
* $\tau_{da} = 5 \cdot 10^7$, $dh = 500$
* $\tau_{da} = 2 \cdot 10^7$, $dh = 250$

Note that the last simulation will take roughly 10 minutes to run. 

**7.2 Create a plot that compares the time series of areal-weighted HRU SWE (the example SWE plot above shows you how to do this) of the four simulations. Create a plot that compares the outlet simulated streamflow for the four simulations. Interpret the results.**

In [ ]:
cases = {1:{'tda':np.log10(10**9),'dh':2000},
         2:{'tda':np.log10(10**8),'dh':1000},
         3:{'tda':np.log10(5*10**7),'dh':500},
         4:{'tda':np.log10(2*10**7),'dh':250}
        }

**7.3 Create a plot that compares the number of HRUs used per simulation to its corresponding SUMMA runtime (documented previously). Assuming the runtime scales linearly with the number of HRUs, provide an estimate of how long it would take to run the "fully-distributed" model (405001 HRUs at ~75 m spatial resolution) over one year. In light of the differences observed between the different model simulations, how should the required increase in computation inform our decision to run a fully distributed model?.**

The most straightforward way to do this is fit a line to the results and then evaluate that function at the number of pixels that would exist in the fully distributed simulation. We provide the template below to fit and evaluate a line. You will need to fill in the actual values.

In [ ]:
nhrus = np.array([n9,n8,n57,n27])
runtime = np.array([r9,r8,r57,r27])
plt.plot(nhrus,runtime,'bo')
f = np.poly1d(np.polyfit(nhrus,runtime,1))
runtime_pred = f(nhrus)
plt.plot(nhrus,runtime_pred,'r')
plt.show()
print('Approximate fully distributed simulation runtime: %d hours' % (f(405001)/3600,))

<br>

# 8. Reflection
You have tested geospatial preprocessing methods to support application of spatially distributed hydrological models. Reflect critically on the limitations of these methods and the benefits of alternative methods for geospatial preprocessing. 250 word maximum.